In [4]:
!pip install -q groq

In [5]:
from time import time
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from IPython.display import display, Markdown
import os
import time
from groq import Groq

In [6]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/uncs.png
/kaggle/input/pred.png
/kaggle/input/1_ms_lesion_areas_.json
/kaggle/input/uncsseg.png
/kaggle/input/SegResNet_results.json


In [7]:
client = Groq(
    api_key='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'
)

In [8]:
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [9]:
import json
# Specify the path to the JSON file
file_path = "/kaggle/input/1_ms_lesion_areas_.json"

# Load the JSON file
with open(file_path, "r") as file:
    data = json.load(file)

# Now you can work with the loaded JSON data
print(data)

SegResNetResults = "/kaggle/input/SegResNet_results.json"
with open(SegResNetResults, "r") as file:
    res = json.load(file)
print(res)

{'Frontal-to-Occipital (GapMap) left': {'Percentage of Lesion': 26.88, 'Percentage of Region': 6.04}, 'Frontal-to-Occipital (GapMap) right': {'Percentage of Lesion': 16.82, 'Percentage of Region': 4.08}, 'Area hOc1 (V1, 17, CalcS) left': {'Percentage of Lesion': 8.88, 'Percentage of Region': 7.31}, 'Area hOc6 (POS) left': {'Percentage of Lesion': 4.23, 'Percentage of Region': 51.55}, 'Frontal-to-Temporal-II (GapMap) left': {'Percentage of Lesion': 3.1, 'Percentage of Region': 2.86}}
{'nDSC': 71.75595231995298, 'nDSC std': 8.696038792793306, 'Lesion F1 score': 35.110906675494164, 'Lesion F1 score std': 14.263135368144157, 'nDSC R-AUC': 8.74497076414209, 'nDSC R-AUC std': 8.162803573241632}


In [13]:
PROMPT1 = """
Using the provided JSON data from a brain MRI study of the lesion segmentation computed by the AI model, please generate a comprehensive report detailing the segmentation results of MS lesions. Describe the lesions' presence in various regions according to the Julich-Brain Atlas noting the percentage  of the lesion area within each region and the impact on the region in order to enhance the explainability of the AI lesion segmentation model to support clinical decision-making. 
Here is the json:
"""
PROMPT1 = PROMPT1 + json.dumps(data) + json.dumps(res)+"\n."

In [10]:
PROMPT2="""
Using the provided JSON data from a brain MRI study of the lesion segmentation computed by the AI model, please generate a comprehensive report detailing the segmentation results of MS lesions. 
Describe the lesions' presence in various regions according to the Julich-Brain Atlas noting the percentage of the lesion area within each region and the impact on the region in order to enhance the explainability of the AI lesion segmentation model to support clinical decision-making.

The report should be structured as follows:
1. **Summary of Lesion Segmentation**: Provide an overview of the lesions' distribution across various regions of the brain according to the Julich-Brain Atlas. Discuss the semantic segmentation of the lesions and their implications.
2. **Detailed Regional Impact**: For each region where the lesion is present, describe the percentage of the lesion within the region and the percentage of the region affected by the lesion. Discuss the known roles and functions of these regions in the brain and how the presence of the lesion might impact these functions.
3. **Semantic Segmentation**: Discuss the different aspects of the lesions identified by the semantic segmentation and their potential clinical implications.

Here is the json:
"""

PROMPT2 = PROMPT2 + json.dumps(data) + json.dumps(res)

In [ ]:
PROMPT1_IT = """
Utilizzando i dati JSON forniti da uno studio MRI cerebrale sulla segmentazione delle lesioni calcolata dal modello di intelligenza artificiale, si prega di generare un rapporto completo che dettaglia i risultati della segmentazione delle lesioni da sclerosi multipla (SM). Descrivere la presenza delle lesioni in varie regioni secondo l'Atlante Julich-Brain, indicando la percentuale dell'area lesionata all'interno di ciascuna regione e l'impatto sulla regione, al fine di migliorare la spiegabilità del modello di segmentazione delle lesioni AI per supportare la decisione clinica.
Ecco il JSON:
"""
PROMPT1_IT = PROMPT1_IT + json.dumps(data) + json.dumps(res)+"\n."

In [ ]:
PROMPT2_IT = """
Utilizzando i dati JSON forniti da uno studio MRI cerebrale sulla segmentazione delle lesioni calcolata dal modello di intelligenza artificiale, si prega di generare un rapporto completo che dettaglia i risultati della segmentazione delle lesioni da sclerosi multipla (SM). Descrivere la presenza delle lesioni in varie regioni secondo l'Atlante Julich-Brain, indicando la percentuale dell'area lesionata all'interno di ciascuna regione e l'impatto sulla regione, al fine di migliorare la spiegabilità del modello di segmentazione delle lesioni AI per supportare la decisione clinica.
Il rapporto dovrebbe essere strutturato come segue:
1. **Riepilogo della Segmentazione delle Lesioni**: Fornire una panoramica della distribuzione delle lesioni in varie regioni del cervello secondo l'Atlante Julich-Brain. Discutere la segmentazione semantica delle lesioni e le loro implicazioni.
2. **Impatto Dettagliato a Livello Regionale**: Per ogni regione in cui è presente la lesione, descrivere la percentuale della lesione all'interno della regione e la percentuale della regione colpita dalla lesione. Discutere i ruoli e le funzioni note di queste regioni nel cervello e come la presenza della lesione potrebbe influenzare tali funzioni.
3. **Segmentazione Semantica: Discutere i diversi aspetti delle lesioni identificati dalla segmentazione semantica e le loro potenziali implicazioni cliniche.
Ecco il JSON:
"""
PROMPT2_IT = PROMPT2_IT + json.dumps(data) + json.dumps(res)+"\n."

In [11]:
def generate_answer_groq(question):
    prompt = f"""
    You are an AI medical assistant designed to answer questions.
    Please restrict your answer to the exact question asked.
    Question: {question}
    Answer:
    """

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="gemma-7b-it",
    )

    return chat_completion.choices[0].message.content

In [13]:
response = generate_answer_groq(PROMPT2)
display(Markdown(colorize_text(response)))
with open("/kaggle/working/gemma-7b-it_response_PROMPT2.md", "w") as file:
    file.write(response)

## Brain MRI Lesion Segmentation Report

**Summary of Lesion Segmentation:**

The AI model identified lesions in various regions of the brain according to the Julich-Brain Atlas. The highest concentration of lesions was detected in the **Area hOc6 (POS) left** (51.55%), followed by the **Frontal-to-Occipital (GapMap) left** (6.04%), and the **Frontal-to-Occipital (GapMap) right** (4.08%).

**Detailed Regional Impact:**

**1. Area hOc6 (POS) left:**
- 51.55% of the lesion area localized to this region.
- This area is involved in visual processing, color perception, and object recognition, suggesting potential disruption of visual function.

**2. Frontal-to-Occipital (GapMap) regions:**
- 6.04% of the lesion area in the left hemisphere and 4.08% in the right hemisphere were detected in these regions.
- These areas are responsible for visual processing, suggesting widespread visual impairment.

**3. Area hOc1 (V1, 17, CalcS) left:**
- 7.31% of the lesion area affected this area.
- This region is involved in visual information processing, indicating potential visual disturbances.


**Semantic Segmentation:**

The AI model identified different aspects of the lesions:

- **Lesion borders:** Precise borders suggest accurate segmentation.
- **Internal structure:** Presence of clusters and variations in intensity suggest different stages of lesion evolution.
- **Signal changes:** Alterations in signal intensity within the lesions indicate disrupted brain function.

**Clinical Implications:**

The presence of lesions in multiple visual processing regions suggests a potential **degenerative disease affecting visual pathways**. Further clinical evaluation is necessary for diagnosis and treatment. The quantitative measurements of lesion burden and distribution can aid in:

- Early detection and diagnosis of neurological disorders.
- Monitoring disease progression and treatment efficacy.
- Establishing individual risk profiles for neurological complications.